# Stint to Previous Stint Comparison

Age, teammates of same nationality, and other variables will be used to predict whether the current stint will have a score higher or lower than the previous stint.

Should we also look at previous stint score?  Will need small feature design

## Model 1 - Random Forest?

## Model 2 - AdaBoost

## Compare Performance

## Hyperparameter Experimentation

## Summary